In [94]:
import socket

In [95]:
port = 33434
address_name = "127.0.0.1"

In [96]:
def get_recv_socket(timeout):
    recv_socket = socket.socket(socket.AF_INET, socket.SOCK_RAW, socket.IPPROTO_ICMP)
    recv_socket.settimeout(timeout)
    return recv_socket
    

In [97]:
def get_send_socket(ttl):
    send_socket = socket.socket(socket.AF_INET, socket.SOCK_DGRAM, socket.IPPROTO_UDP)
    send_socket.setsockopt(socket.IPPROTO_HOPOPTS, socket.IP_TTL, ttl)
    return send_socket

In [98]:
def get_address(dest_addr, recv_socket, send_socket):
    address_name, responce_ip = None, "0.0.0.0"
    try:
        # Отправка пакета
        send_socket.sendto(b"Hello", (dest_addr, port))
        data, address = recv_socket.recvfrom(1024) #получаем ответ
        responce_ip = address[0]
        try:
            address_name = socket.gethostbyaddr(address[0])[0]

        except socket.error:
            address_name = address[0]

    except socket.error as err:
        print('-', end=' ')
    return address_name, responce_ip

In [99]:
def trace_udp(dest_name, ttl, timeout=3, att=4):
    global address_name
    recv_socket = get_recv_socket(timeout)
    send_socket = get_send_socket(ttl)
    dest_addr = socket.gethostbyname(dest_name)
    for i in range(att):
        address_name, responce_ip = get_address(dest_addr, recv_socket, send_socket)
        if (not(address_name == None and responce_ip == "0.0.0.0")): break
    print()
    
    send_socket.close()
    recv_socket.close()

    return address_name, responce_ip

In [100]:
def tracert(dest_name, timeout=3, max_steps=128, att=4):
    print(f"Start tracing: {dest_name}, timeout: {timeout}, max steps: {max_steps}")
    curr = 1
    while(curr <= max_steps):
        answ, answ_ip = trace_udp(dest_name, curr, timeout)
        if answ == answ_ip:
            print(answ)
        elif answ_ip == "0.0.0.0":
            curr+=1
            continue
        else:
            print(answ, answ_ip)

        if (answ == dest_name):
            print('Reached ')
            break
        curr+= 1

In [101]:
tracert("ya.ru")


Start tracing: ya.ru, timeout: 3, max steps: 128

172.19.2.1
- - - - 
- - - - 

sas-32z5-ae2.yndx.net 87.250.239.203
- - - - 

ya.ru 87.250.250.242
Reached 
